# Quick Reference

## 0. Setup
Please make sure your environment is set up according to the [instructions here](https://nasa-navo.github.io/navo-workshop/00_SETUP.html).

Ensure you have the latest version of the workshop material by updating your environment:
TBD

## 1. Overview
NASA services can be queried from Python in multiple ways.
* Generic Virtual Observatory (VO) queries.
  * Call sequence is consistent, including for non-NASA resources.
  * Use the [`pyvo` package](https://pyvo.readthedocs.io/en/latest/).
  * [Known issues/caveats](https://github.com/NASA-NAVO/navo-workshop/blob/main/KNOWN_ISSUES.md).
* [Astroquery](https://astroquery.readthedocs.io/en/latest/) interfaces:
  * Call sequences not quite as consistent, but follow similar patterns.
* Ad hoc archive-specific interfaces

## 2. VO Services
This workshop will introduce 4 types of VO queries:
* **VO Registry** - Discover what services are available worldwide
* **Simple Cone Search** - Search for catalog object within a specified cone region
* **Simple Image Access** - Search for image products within a spatial region
* **Simple Spectral Access** - Search for spectral products within a spatial region
* **Table Access** - SQL-like queries to databases

### 2.0 Import Necessary Packages

In [1]:
# Generic VO access routines
import pyvo as vo

# For specifying coordinates and angles
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy import units as u

# For downloading files
from astropy.utils.data import download_file

# Ignore unimportant warnings
import warnings
warnings.filterwarnings('ignore', '.*Unknown element mirrorURL.*', vo.utils.xml.elements.UnknownElementWarning)

### 2.1 Look Up Services in VO Registry
Simple example:  Find Simple Cone Search (conesearch) services related to SWIFT.

In [2]:
services = vo.regsearch(servicetype='conesearch', keywords=['swift'])
services

<Table length=207>
                 ivoid                  ...       intf_roles     
                                        ...                      
                 object                 ...         object       
--------------------------------------- ... ---------------------
ivo://archive.stsci.edu/catalogs/mastcs ...                   std
    ivo://bsdc.icranet.org/sds82/q/cone ...                   std
               ivo://cds.vizier/b/swift ...                   std
                ivo://cds.vizier/ii/339 ... std:::py VO sep:::std
                 ivo://cds.vizier/ix/43 ... std:::py VO sep:::std
                 ivo://cds.vizier/ix/51 ...                   std
                 ivo://cds.vizier/ix/58 ... std:::py VO sep:::std
          ivo://cds.vizier/j/a+a/427/87 ...                   std
          ivo://cds.vizier/j/a+a/448/l9 ...                   std
                                    ... ...                   ...
          ivo://nasa.heasarc/swiftmastr ...              

#### 2.1.1 Use different arguments/values to modify the simple example
| Argument | Description | Examples |
| :-----: | :----------- | :-------- |
| **servicetype** | Type of service | `conesearch` or `scs` for **Simple Cone Search**<br> `image` or `sia` for **Simple Image Access**<br> `spectrum` or `ssa` for **Simple Spectral Access**<br> `table` or `tap` for **Table Access Protocol**|
| **keyword** | List of one or more keyword(s) to match service's metadata. Both ORs and ANDs may be specified.<br><ul><li>(OR) A list of keywords match a service if **any** of the keywords match the service.</li><li>(AND) If a  keyword contains multiple space-delimited words, **all** the words must match the metadata.</li></ul>| `['galex', 'swift']` matches 'galex' or 'swift'<br>`['hst survey']` matches services mentioning both 'hst' and 'survey' |
| **waveband** | Resulting services have data in the specified waveband(s) | ‘radio’, ‘millimeter’, ‘infrared’, ‘optical’, ‘uv’, ‘euv’, ‘x-ray’ ‘gamma-ray’ |

#### 2.1.2 Inspect the results.
##### Using pyvo
Although not lists, `pyvo` results can be iterated over to see each individual result.  The results are specialized based on the type of query, providing access to the important properties of the results.  Some useful accessors with registry results are:
* `short_name` - A short name
* `res_title` - A more descriptive title
* `res_description` - A more verbose description
* `reference_url` - A link for more information
* `ivoid` - A unique identifier for the service.  Gives some indication of what organization is serving the data.

In [3]:
# Print the number of results and the 1st 4 short names and titles.
print(f'Number of results: {len(services)}\n')
for s in list(services)[:4]:  # (Treat services as list to get the subset of rows)
    print(f'{s.short_name} - {s.res_title}')

Number of results: 207

MAST CS - MAST ConeSearch
sds82 cone - SDS82 catalog
B/swift - Swift Master Catalog
II/339 - Swift/UVOT Serendipitous Source Catalog


##### Filtering results
Of the services we found, which one(s) have 'stsci.edu' in their unique identifier?

In [4]:
stsci_services = [s for s in services if 'stsci.edu' in s.ivoid]
for s in stsci_services:
    print (f'(STScI): {s.short_name} - {s.res_title}')

(STScI): MAST CS - MAST ConeSearch


##### Using astropy
With the `to_table()` method, `pyvo` results can also be converted to Astropy `Table` objects which offer a variety of addional features. See [astropy.table](http://docs.astropy.org/en/stable/table/) for more on working with Astropy Tables.

In [5]:
# Convert to an Astropy Table
services_table = services.to_table()

# Print the column names and display 1st 3 rows with a subset of columns
print(f'\nColumn Names:\n{services_table.colnames}\n')  
services_table['short_name', 'res_title', 'res_description'][:3]  


Column Names:
['ivoid', 'res_type', 'short_name', 'res_title', 'content_level', 'res_description', 'reference_url', 'creator_seq', 'content_type', 'source_format', 'source_value', 'region_of_regard', 'waveband', 'access_urls', 'standard_ids', 'intf_types', 'intf_roles']



short_name,res_title,res_description
object,object,object
MAST CS,MAST ConeSearch,"All MAST catalog holdings are available via a ConeSearch endpoint. \nThis service provides access to all, with an optional non-standard parameter for an individual catalog to query. \nThe available missions are listed at http://archive.stsci.edu/vo/mast_services.html, \nand include Hubble (HST) data, Kepler, K2, IUE, HUT, EUVE, FUSE, UIT, WUPPE, BEFS, TUES, IMAPS, High Level Science Products (HLSP), Copernicus, HPOL, VLA First, XMM-OM, and SWIFT."
sds82 cone,SDS82 catalog,The SDS82 catalog is the result of the Swift DeepSky pipeline over\nall observations made by Swift-XRT within the Stripe82.
B/swift,Swift Master Catalog,(no description available)


### 2.2 Cone search
Example:  Find a cone search service for the USNO-B catalog and search it around M51 with a .1 degree radius.  (More inspection could be done on the service list instead of blindly choosing the first service.)  

The position (`pos`) is best specified with [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) objects. 

The size of the region is specified with the `radius` keyword and may be decimal degrees or an Astropy [Angle](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html#astropy.coordinates.Angle).

In [6]:
m51_pos = SkyCoord.from_name("m51")
services = vo.regsearch(servicetype='conesearch', keywords='usno-b')
results = services[0].search(pos=m51_pos, radius=0.1)
# Astropy Table is useful for displaying cone search results.
results.to_table()  

ObjID,Zone,SeqNo,RA,DEC,pmRA,pmDEC,e_pmRA,e_pmDEC,e_RA,e_DEC,Epoch,B1Mag,R1s_g,B2Mag,B2s_g,R2Mag,R2s_g,NMag,mag,B1s_g,R1Mag,distance
,,,deg,deg,mas / yr,mas / yr,mas / yr,mas / yr,arcsec,arcsec,yr,mag,,mag,,mag,,mag,mag,,mag,arcsec
int64,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,int32,float32,int32,float32,float32,int32,float32,float32
5888400445251,1371,282435,202.472919444444,47.1954444444444,2.081668e-13,2.081668e-13,0.0,0.0,999.0,999.0,1981.8,-999999500.0,1,9.4,9,-999999500.0,0,7.86,-999999500.0,0,8.98,0.13680981
5892695420475,1372,290363,202.494625,47.2065472222222,2.081668e-13,2.081668e-13,0.0,0.0,332.0,999.0,1956.2,20.13,3,-999999500.0,0,-999999500.0,0,-999999500.0,-999999500.0,4,15.79,1.225397
5892695420458,1372,290346,202.436875,47.2094416666667,2.081668e-13,2.081668e-13,0.0,0.0,999.0,359.0,1976.2,-999999500.0,1,-999999500.0,0,-999999500.0,0,10.89,-999999500.0,0,13.27,1.5814886
5892695420468,1372,290356,202.469830555556,47.2232805555556,2.081668e-13,2.081668e-13,0.0,0.0,560.0,934.0,1976.2,-999999500.0,3,-999999500.0,0,-999999500.0,0,18.4,-999999500.0,0,17.78,1.6813676
5888400445256,1371,282440,202.480638888889,47.1682194444444,-20.0,-6.0,5.0,5.0,205.0,201.0,1969.5,12.79,8,-999999500.0,0,-999999500.0,0,14.87,-999999500.0,1,15.4,1.6839039
5892695420469,1372,290357,202.4706,47.2235555555556,2.081668e-13,2.081668e-13,0.0,0.0,709.0,130.0,1956.2,15.89,3,-999999500.0,0,-999999500.0,0,-999999500.0,-999999500.0,1,17.68,1.6983492
5892695420470,1372,290358,202.474338888889,47.2264416666667,2.081668e-13,2.081668e-13,0.0,0.0,200.0,490.0,1956.2,13.49,4,-999999500.0,0,-999999500.0,0,-999999500.0,-999999500.0,1,18.74,1.8810501
5888400445204,1371,282388,202.423558333333,47.1977666666667,2.081668e-13,2.081668e-13,0.0,0.0,466.0,480.0,1969.5,15.66,8,-999999500.0,0,-999999500.0,0,14.97,-999999500.0,1,15.22,1.8820878


### 2.3 Image search
Example:  Find an image search service for GALEX, and search it around coordinates 13:37:00.950,-29:51:55.51 (M83) with a radius of .2 degrees.  Download the first file in the results.
#### Find an image service

In [7]:
services = vo.regsearch(servicetype='image', keywords=['galex'])
services.to_table()['ivoid', 'short_name', 'res_title']

ivoid,short_name,res_title
object,object,object
ivo://archive.stsci.edu/sia/galex,GALEX,Galaxy Evolution Explorer (GALEX)
ivo://mast.stsci/siap/galex_atlas,GALEX_Atlas,GALEX Atlas of Nearby Galaxies
ivo://nasa.heasarc/skyview/galex,GALEX,Galaxy Explorer All Sky Survey: Near UV


#### Search one of the services
The first service looks good.  Search it!

For more details on using `SkyCoord`, see [its documentation](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord)

**NOTE**:  For image searches, the size of the region is defined by the `size` keyword which is more like a diameter than a radius.

In [8]:
m83_pos = SkyCoord('13h37m00.950s -29d51m55.51s')
results = services[1].search(pos=m83_pos, size=.2)

# We can look at the results.
results.to_table()

filename,id,ra_j2000,dec_j2000,url,filesize,mjdmean,naxes,naxis,scale,cd,format,ref_frame,equinox,coord_projection,crpix,crval,ctype,bandpass_id,bandpass_refvalue,bandpass_unit,bandpass_hilimit,bandpass_lolimit,processing,project,preview,representative,object_id
,,deg,deg,,byte,d,,pix,deg / pix,deg / pix,,,yr,,pix,pix,,,m,m,m,m,,,,,
object,object,float64,float64,object,int32,float64,int32,object,object,object,object,object,float32,str3,object,object,object,object,float64,object,float64,float64,object,object,object,object,object
h_m83-fd-int.fits.gz,MAST.GALEX,204.25375,-29.86575,http://archive.stsci.edu/pub/hlsp/galex_atlas/h_m83-fd-int.fits.gz,13032100,--,2,[1805 1805],[-0.0004166667 0.0004166667],[-0.0004166667 0.0004166667],image/fits,FK5,2000.0,TAN,[1889.5 366.5],[203.78 -30.0903],RA---TAN DEC--TAN,GALEX FUV,1516.0,meters,1350.0,1750.0,Z,galex_atlas,,n,MESSIER083
h_m83-nd-int.fits.gz,MAST.GALEX,204.25375,-29.86575,http://archive.stsci.edu/pub/hlsp/galex_atlas/h_m83-nd-int.fits.gz,13032100,--,2,[1805 1805],[-0.0004166667 0.0004166667],[-0.0004166667 0.0004166667],image/fits,FK5,2000.0,TAN,[1889.5 366.5],[203.78 -30.0903],RA---TAN DEC--TAN,GALEX NUV,2267.0,meters,1750.0,2800.0,Z,galex_atlas,,n,MESSIER083


#### Download an image
For the first result, print the file format and download the file. If repeatedly executing this code, add `cache=True` to `download_file()` to prevent repeated downloads.

See [`download_file()` documentation here.](https://docs.astropy.org/en/stable/api/astropy.utils.data.download_file.html#astropy.utils.data.download_file)

In [9]:
print(results[0].format)
file_name = download_file(results[0].getdataurl())  
file_name

image/fits


'/tmp/astropy-download-1995-79gj3juv'

### 2.4 Spectral search
Example:  Find a spectral service for x-ray data.  Query it around Delta Ori with a search **diameter** of 10 arc minutes, and download the first data product.  Note that the results table can be inspected for potentially useful columns.

Spectral search is very similar to image search. In this example, note:
* **`diameter`** defines the size of the search region
* `waveband` used in `regsearch()`
* Astropy `Angle` used to specify radius units other than degrees.

In [10]:
# Search for a spectrum search service that has x-ray data.
services = vo.regsearch(servicetype='spectrum', waveband='x-ray')

# Assuming there are services and the first one is OK...
results = services[0].search(pos=SkyCoord.from_name("Delta Ori"), 
                             diameter=Angle(10 * u.arcmin))

# Assuming there are results, download the first file.
print(f'Title: {results[0].title}, Format: {results[0].format}')
file_name = download_file(results[0].getdataurl())  
file_name

Title: acisf00639N004_pha2.fits, Format: application/fits


'/tmp/astropy-download-1995-kk3bpcpi'

### 2.5 Table search
Example:  Find the HEASARC Table Access Protocol (TAP) service, get some information about the available tables.

In [11]:
services = vo.regsearch(servicetype='tap', keywords=['heasarc'])
print(f'{len(services)} service(s) found.')
# We found only one service.  Print some info about the service and its tables.
print(f'{services[0].describe()}')
tables = services[0].service.tables  # Queries for details of the service's tables
print(f'{len(tables)} tables:')
for t in tables:
    print(f'{t.name:30s} - {t.description}')  # A more succinct option than t.describe()

1 service(s) found.
HEASARC Xamin Catalog Interface
Short Name: HEASARC
IVOA Identifier: ivo://nasa.heasarc/services/xamin
Access modes: tap
Base URL: https://heasarc.gsfc.nasa.gov/xamin/vo/tap

The HEASARC is NASA domain archive for high-energy and microwave astronomy.
The Xamin interface provides access to over 600 observation and object tables.
This includes observation tables for more than 30 missions and observatories
and hundreds of derived object tables. Non-high energy tables are included to
make it easier for users to compare information.

Subjects: HEASARC
Waveband Coverage:
None


1022 tables:
TAP_SCHEMA.columns             - Available columns
TAP_SCHEMA.key_columns         - Key/Foreign key pairs -- usused
TAP_SCHEMA.keys                - Joins between tables -- unused
TAP_SCHEMA.schemas             - Available schemas
TAP_SCHEMA.tables              - Available tables
"first"                        - Faint Images of the Radio Sky at Twenty cm (FIRST)
a1                             - HEAO 1 A1 X-Ray Source Catalog
a1point                        - HEAO 1 A1 Lightcurves
a2lcpoint                      - HEAO 1 A2 Pointed Lightcurves
a2lcscan                       - HEAO 1 A2 Scanned Lightcurves
a2led                          - HEAO 1 A2 LED Catalog
a2pic                          - HEAO 1 A2 Piccinotti Catalog
a2point                        - HEAO 1 A2 Pointing Catalog
a2rtraw                        - HEAO 1 A2 Raw Rates
a2specback                     - HEAO 1 A2 Spectra Background
a2spectra                      - HEAO 1 A2 Spectra
a3                             - H

vlacos324m                     - VLA-COSMOS Survey 324-MHz Continuum Source Catalog
vlacos3ghz                     - VLA-COSMOS 3-GHz Large Project Source Catalog
vlacosmjsc                     - VLA-COSMOS Project 1.4-GHz Joint Source Catalog
vlacosmos                      - VLA-COSMOS Large Project 1.4-GHz Source Catalog
vlacosxoid                     - VLA-COSMOS 3-GHz Large Project Multiwavelength Counterparts Cata
vlaecdfs1p4                    - VLA Extended-Chandra Deep Field-South 1.4-GHz Source Catalog
vlaecdfscls                    - VLA Extended-Chandra Deep Field-South Classification Catalog
vlaecdfsoi                     - VLA Extended-Chandra Deep Field-South 1.4-GHz Sources Opt/IR Cou
vlaen20cm                      - VLA ELAIS N1, N2, N3 Fields 20-cm Source Catalog
vlagbsoph                      - VLA Goulds Belt Survey Ophiuchus Complex Source Catalog
vlagbsori                      - VLA Goulds Belt Survey Orion Complex Source Catalog
vlagbsper                      - VL

#### Column Information
For any table, we can list the column names and descriptions.

In [12]:
for c in tables['zcat'].columns:
    print(f'{c.name:30s} - {c.description}')

comments                       - Comments
"__row"                        - System index column
diameter_2                     - Diameter 2 (arcmin)
structure                      - Structure Flag
"distance"                     - Distance Determined by Independent Source (Mpc)
notes                          - Diameter (arcmin) or Spectrum ID
class                          - Browse Object Classification
lii                            - Galactic Longitude
bii                            - Galactic Latitude
"__y_ra_dec"                   - System unit vector column
ugc_or_eso                     - UGC or ESO Number
morph_type                     - Morphological (T) Type
ref_bmag                       - Reference Code for B Magnitude
name                           - Catalog Designation
radial_velocity_error          - Radial Velocity Error
rfn_number                     - Plate Number
ref_radial_velocity            - Reference Code for Radial Velocity
"__x_ra_dec"                   - System 

#### Perform a Query
Example:  Perform a cone search on the ZCAT catalog at M83 with a 1.0 degree radius.

In [13]:
coord = SkyCoord.from_name("m83")
query = f'''
SELECT ra, dec, Radial_Velocity, radial_velocity_error, bmag, morph_type FROM public.zcat as cat where 
contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{coord.ra.deg},{coord.dec.deg},1.0))=1
'''
results = services[0].service.run_async(query)

results.to_table()

ra,dec,radial_velocity,radial_velocity_error,bmag,morph_type
deg,deg,km / s,km / s,,
float64,float64,int32,int16,float32,int16
204.79302,-30.74322,4795,--,16.70,--
203.96838,-30.43180,3847,53,--,--
203.59607,-30.39890,10310,300,--,--
203.95738,-30.38237,4196,--,16.40,--
203.78459,-30.11771,300,--,16.70,--
204.36462,-30.03935,14985,--,18.00,--
203.23058,-29.91932,300,--,16.60,--
204.25515,-29.87123,518,5,8.22,5


## 3. Astroquery 
Many archives have Astroquery or Astroquery-compliant modules for data access, including:

* Astroquery
  * [HEASARC Queries (astroquery.heasarc)](https://astroquery.readthedocs.io/en/latest/heasarc/heasarc.html)
  * [HITRAN Queries (astroquery.hitran)](https://astroquery.readthedocs.io/en/latest/hitran/hitran.html)
  * [IRSA Image Server program interface (IBE) Queries (astroquery.ibe)](https://astroquery.readthedocs.io/en/latest/ibe/ibe.html)
  * [IRSA Queries (astroquery.ipac.irsa)](https://astroquery.readthedocs.io/en/latest/ipac/irsa/irsa.html)
  * [IRSA Dust Extinction Service Queries (astroquery.ipac.irsa.irsa_dust)](https://astroquery.readthedocs.io/en/latest/ipac/irsa/irsa_dust.html)
  * [JPL Spectroscopy Queries (astroquery.jplspec)](https://astroquery.readthedocs.io/en/latest/jplspec/jplspec.html)
  * [MAST Queries (astroquery.mast)](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
  * [NASA ADS Queries (astroquery.nasa_ads)](https://astroquery.readthedocs.io/en/latest/nasa_ads/nasa_ads.html)
  * [NED Queries (astroquery.ipac.ned)](https://astroquery.readthedocs.io/en/latest/ipac/ned/ned.html)
* Astroquery-compliant
  * [KOA Queries (pykoa.koa)](https://koa.ipac.caltech.edu/UserGuide/PyKOA/PyKOA.html)

For more, see https://astroquery.readthedocs.io/en/latest/

### 3.1 NED
Example:  Get an Astropy Table containing the objects from paper 2018ApJ...858...62K.  For more on the API, see [astroquery](https://astroquery.readthedocs.io/en/latest/ipac/ned/ned.html)

In [14]:
from astroquery.ipac.ned import Ned
objects_in_paper = Ned.query_refcode('2018ApJ...858...62K')
objects_in_paper

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,str30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,WISEA J115844.06+273505.3,179.68371,27.585,G,106.0,0.000354,SLS,17.2g,--,27,0,22,12,7,0,0
2,KUG 1207+367,182.48529,36.43433,G,341.0,0.001137,SUN,15.5,--,38,0,28,17,7,4,0
3,NGC 4244,184.37358,37.80711,G,244.0,0.000815,SUN,10.88,--,469,15,94,52,25,7,1
4,SDSS J121731.98+323157.7,184.38329,32.53271,G,447.0,0.001492,SLS,17.95,--,6,0,15,6,2,4,0
5,SDSS J121811.04+465501.2,184.54614,46.91685,G,402.0,0.001341,SLS,17.75,--,20,0,37,18,11,4,0
6,VCC 0530,185.53154,15.79911,G,1299.0,0.004333,SUN,15.8,--,67,1,20,23,13,3,3
7,PGC1 0040692 NED033,186.37146,28.48244,G,483.0,0.001611,SUN,16.47,--,39,0,7,14,8,0,0
8,UGC 07512,186.42223,2.15898,G,1505.0,0.00502,SUN,15.00,--,64,1,21,17,12,4,0
